In [0]:
# -----------------------------
# SILVER NOTEBOOK
# -----------------------------

print("This is Silver Notebook")

dbutils.widgets.text("run_date", "2026-01-15")
run_date = dbutils.widgets.get("run_date")
print("Silver running for date:", run_date)

# READ BRONZE TABLE
bronze_df = spark.table("default.bronze_movies")

from pyspark.sql.functions import col, when

silver_df = (
    bronze_df
    .filter(col("title").isNotNull())
    .filter(col("industry").isNotNull())
    .filter(col("studio").isNotNull())
    .filter(col("budget").isNotNull() & (col("budget") >= 0))
    .filter(col("revenue").isNotNull() & (col("revenue") >= 0))
    .withColumn(
        "imdb_rating_clean",
        when(col("imdb_rating") == "NULL", None)
        .otherwise(col("imdb_rating").cast("double"))
    )
    .dropDuplicates()
)

silver_df.show(5)

# SAVE SILVER TABLE (schema migration enabled)
silver_df.write \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("default.silver_movies")


This is Silver Notebook
Silver running for date: 2026-01-15
+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+-----------------+
|               title| industry|release_year|       imdb_rating|              studio|  budget|             revenue|     unit|currency|language|      ingestion_time|imdb_rating_clean|
+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+-----------------+
|     Pather Panchali|Bollywood|        1955|8.3000000000000007|Government of Wes...|70000.00|100000.0000000000...|Thousands|     INR| Bengali|2026-01-15 05:42:...|              8.3|
|Doctor Strange in...|Hollywood|        2022|                 7|      Marvel Studios|  200.00|  954.80000000000000| Millions|     USD| English|2026-01-15 05:42:...|              7.0|
|Thor: The Dark Wo...|Hol